# Search Engine Project

Αυτό το notebook περιγράφει τη δημιουργία μιας απλής μηχανής αναζήτησης, βασισμένη στα βήματα της εργασίας.

## Βήμα 1: Συλλογή Δεδομένων

Το πρώτο βήμα περιλαμβάνει τη συλλογή δεδομένων. Αρχικά, σχεδιάστηκε ένας crawler για την αυτόματη συλλογή άρθρων από το Wikipedia. Ωστόσο, λόγω πιθανών προβλημάτων (π.χ., προσωρινό αποκλεισμό της διεύθυνσης IP) και για λόγους πρακτικότητας, χρησιμοποιήθηκε το dataset `goalscorers.csv` από το Kaggle.

Το dataset περιέχει δεδομένα για σκόρερ ποδοσφαιρικών αγώνων, και μετατράπηκε σε JSON μορφή (`goalscorers_processed.json`), μέσω online εργαλείου για να χρησιμοποιηθεί στη μηχανή αναζήτησης.

Παρακάτω μπορείτε να παρατίθεται ο κώδικας του crawler (ενδεικτικός):

### Εισαγωγή Crawler (Κώδικας):


```
import requests
from bs4 import BeautifulSoup
import json
import time
import random

# Αρχική Διεύθυνση Wikipedia
random_article_url = "https://en.wikipedia.org/wiki/Special:Random"

# Συνάρτηση για λήψη περιεχομένου από Wikipedia
def fetch_article_content(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            parser = BeautifulSoup(response.text, 'html.parser')
            title = parser.find('h1', {'id': 'firstHeading'}).text.strip()
            content_section = parser.find('div', {'class': 'mw-parser-output'})
            paragraphs = content_section.find_all('p')
            content = "\n".join([para.text for para in paragraphs if para.text.strip()])
            return {"title": title, "url": response.url, "content": content}
        else:
            print(f"Failed to fetch page: {url} (Status code: {response.status_code})")
    except Exception as error:
        print(f"Error during fetching: {url} - {error}")
    return None

# Διαδικασία συλλογής άρθρων
def gather_articles(limit=100):
    collected_articles = []
    visited_urls = set()

    while len(collected_articles) < limit:
        print(f"Fetching article {len(collected_articles) + 1} of {limit}...")
        article_data = fetch_article_content(random_article_url)

        if article_data and article_data['url'] not in visited_urls:
            collected_articles.append(article_data)
            visited_urls.add(article_data['url'])
            print(f"Added: {article_data['title']}")
        else:
            print("Duplicate or invalid article, retrying...")

        time.sleep(random.uniform(1.5, 3.5))  # Τυχαία καθυστέρηση

    return collected_articles

# Αποθήκευση άρθρων σε αρχείο JSON
def save_articles_to_file(articles, filename="articles_dataset.json"):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(articles, file, ensure_ascii=False, indent=4)
    print(f"Successfully saved {len(articles)} articles to {filename}")

# Εκτέλεση Προγράμματος
if __name__ == "__main__":
    total_articles = 150  # Αριθμός άρθρων προς συλλογή
    articles_data = gather_articles(limit=total_articles)
    save_articles_to_file(articles_data)
```

### Ενδεικτικά Αποτελέσματα Χρήσης του Crawler

Παρακάτω παρατίθενται τα αποτελέσματα από τη λειτουργία του crawler, χωρίς επιπλέον τροποποίηση. Αυτά τα αποτελέσματα καταγράφουν τους τίτλους και τις διευθύνσεις URL άρθρων από το Wikipedia, πριν διακόψουμε την εκτέλεση, ώστε να αποφευχθούν τα προαναφερόμενα πιθανά προβλήματα, που ίσως προέκυπταν και αφορούν στη διεύθυνση IP:

```
PS C:\Users\johnp\Desktop\py\AnaktisiPliroforias> & C:/Users/johnp/AppData/Local/Microsoft/WindowsApps/python3.11.exe c:/Users/johnp/Desktop/py/AnaktisiPliroforias/crawler.py
Fetching article 1 of 150...
Added: Tiro al piccione
Fetching article 2 of 150...
Added: LGBTQ theatre
Fetching article 3 of 150...
Added: Wickenburg, Arizona
Fetching article 4 of 150...
Added: 1616 in music
Fetching article 5 of 150...
Added: Epithelial cell adhesion molecule
Fetching article 6 of 150...
Added: Jerzy Olesiak
Fetching article 7 of 150...
Added: Miss Teen USA 1995
Fetching article 8 of 150...
Added: International Tuba Euphonium Association
Traceback (most recent call last):
  File "c:\Users\johnp\Desktop\py\AnaktisiPliroforias\crawler.py", line 56, in <module>
    articles_data = gather_articles(limit=total_articles)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\johnp\Desktop\py\AnaktisiPliroforias\crawler.py", line 43, in gather_articles
    time.sleep(random.uniform(1.5, 3.5))  # Τυχαία καθυστέρηση
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
```

## Εισαγωγή Δεδομένων

Για την προεπεξεργασία κειμένου, χρησιμοποιούμε τα δεδομένα που περιλαμβάνονται στο αρχείο `goalscorers_processed.json`. Το αρχείο περιέχει πληροφορίες για σκόρερ ποδοσφαιρικών αγώνων, με τις ακόλουθες βασικές δομές:

1. `date`: Η ημερομηνία του αγώνα.
2. `home_team`, `away_team`: Οι ομάδες του αγώνα.
3. `scorer`: Το όνομα του σκόρερ.
4. `team`: Η ομάδα του σκόρερ.
5. `minute`: Το λεπτό του γκολ.
6. `own_goal`, `penalty`: Πληροφορίες για αυτογκόλ ή πέναλτι.

Παρακάτω φορτώνουμε το αρχείο JSON ώστε να χρησιμοποιηθεί στη συνέχεια:


In [1]:
# Φόρτωση JSON αρχείου
import json

try:
    with open("goalscorers_processed.json", "r", encoding="utf-8") as file:
        data = json.load(file)
    print(f"Φορτώθηκαν {len(data)} εγγραφές.")
except FileNotFoundError:
    print("Το αρχείο goalscorers_processed.json δεν βρέθηκε. Βεβαιωθείτε ότι είναι στο ίδιο directory με το Notebook.")


Το αρχείο goalscorers_processed.json δεν βρέθηκε. Βεβαιωθείτε ότι είναι στο ίδιο directory με το Notebook.


### Σημαντική Σημείωση
Για να λειτουργήσει το Notebook, βεβαιωθείτε ότι το αρχείο `goalscorers_processed.json` βρίσκεται στον ίδιο φάκελο με αυτό το Notebook. Το αρχείο περιέχει τα δεδομένα που χρησιμοποιούνται στα επόμενα βήματα.

## Βήμα 2: Προεπεξεργασία Κειμένου

Σε αυτό το βήμα, εφαρμόζουμε προεπεξεργασία κειμένου στα δεδομένα μας, ώστε να είναι έτοιμα για τη δημιουργία ευρετηρίου και τη διαδικασία αναζήτησης. Η προεπεξεργασία περιλαμβάνει:

1. **Tokenization**: Διαχωρισμός του κειμένου σε λέξεις.
2. **Αφαίρεση Stopwords**: Αφαίρεση κοινών λέξεων που δεν προσφέρουν πληροφορία (π.χ., "και", "το").
3. **Lemmatization ή Stemming**: Κανονικοποίηση λέξεων στην αρχική τους μορφή.

Χρησιμοποιούμε το αρχείο `text_processing.py` για να εφαρμόσουμε αυτές τις λειτουργίες.

In [2]:
import json
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

# Κατεβάζουμε τα απαραίτητα πακέτα NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Φόρτωση JSON αρχείου
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Συνάρτηση προεπεξεργασίας κειμένου
def preprocess_text(text):
    if not isinstance(text, str):
        return text  # Επιστρέφει το αρχικό αν δεν είναι string

    # Αφαίρεση ειδικών χαρακτήρων (εκτός κενών και παύλας)
    text = re.sub(r'[^\w\s-]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Αφαίρεση stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]

    # Lemmatization (Διατήρηση κεφαλαίων)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return " ".join(tokens)  # Επιστροφή επεξεργασμένου κειμένου ως string

# Επεξεργασία όλων των πεδίων του dataset
def preprocess_dataset(dataset):
    for entry in dataset:
        for key in entry:
            if isinstance(entry[key], str):  # Επεξεργασία μόνο για string
                entry[key] = preprocess_text(entry[key])
            elif isinstance(entry[key], bool):  # Διατήρηση των boolean τιμών
                entry[key] = entry[key]
    return dataset

# Αποθήκευση JSON ως έγκυρο JSON αρχείο (λίστα εγγραφών)
def save_json_inline(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        # Αποθηκεύει όλες τις εγγραφές ως μία λίστα
        json.dump(data, file, ensure_ascii=False, indent=4)  # Χρήση indent για πιο αναγνώσιμο JSON

# Αποθήκευση JSON σε μορφή μίας γραμμής ανά εγγραφή
#def save_json_inline(data, file_path):
#    with open(file_path, 'w', encoding='utf-8') as file:
#        for entry in data:
#            # Μετατροπή της εγγραφής σε JSON string μίας γραμμής
#            json_line = json.dumps(entry, ensure_ascii=False)
#            file.write(json_line + '\n')  # Προσθέτουμε νέα γραμμή μετά από κάθε εγγραφή

# Κύριο πρόγραμμα
input_file = 'goalscorers.json'  # Διαδρομή εισόδου
output_file = 'goalscorers_processed.json'  # Διαδρομή εξόδου

# Φόρτωση δεδομένων
data = load_json(input_file)

# Επεξεργασία δεδομένων
processed_data = preprocess_dataset(data)

# Αποθήκευση δεδομένων σε γραμμική μορφή
save_json_inline(processed_data, output_file)

print(f"Η αποθήκευση ολοκληρώθηκε. Το αρχείο αποθηκεύτηκε στη διαδρομή: {output_file}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\johnp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Η αποθήκευση ολοκληρώθηκε. Το αρχείο αποθηκεύτηκε στη διαδρομή: goalscorers_processed.json


## Βήμα 3: Δημιουργία Ευρετηρίου (Indexing)

Σε αυτό το βήμα δημιουργούμε ένα ανεστραμμένο ευρετήριο (inverted index) για τα δεδομένα μας. Το ευρετήριο επιτρέπει τη γρήγορη αναζήτηση και ανάκτηση πληροφοριών, βασισμένη στις λέξεις-κλειδιά.

Η διαδικασία περιλαμβάνει:
1. **Φόρτωση των επεξεργασμένων δεδομένων** από το αρχείο `goalscorers_processed.json`.
2. **Δημιουργία του ευρετηρίου**:
   - Καταχώρηση των όρων που εμφανίζονται σε κάθε εγγραφή (π.χ., scorer).
3. **Αποθήκευση του ευρετηρίου** σε αρχείο `inverted_index.json`.

In [3]:
import json
from collections import defaultdict

# Φόρτωση του JSON αρχείου
with open("goalscorers_processed.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Δημιουργία ανεστραμμένου πίνακα
inverted_index = defaultdict(list)

for idx, record in enumerate(data):
    scorer = record["scorer"]
    team = record["team"]
    match_key = f"{record['date']} - {record['home_team']} vs {record['away_team']}"

    # Προσθήκη στο ανεστραμμένο ευρετήριο
    inverted_index[scorer].append({
        "team": team,
        "match": match_key,
        "minute": record["minute"],
        "own_goal": record["own_goal"],
        "penalty": record["penalty"]
    })

# Αποθήκευση του ανεστραμμένου πίνακα σε αρχείο
with open("inverted_index.json", "w", encoding="utf-8") as outfile:
    json.dump(inverted_index, outfile, ensure_ascii=False, indent=4)

print("Ο ανεστραμμένος πίνακας δημιουργήθηκε και αποθηκεύτηκε στο inverted_index.json.")

Ο ανεστραμμένος πίνακας δημιουργήθηκε και αποθηκεύτηκε στο inverted_index.json.


## Βήμα 4: Επεξεργασία Ερωτήματος και Κατάταξη Αποτελεσμάτων

Σε αυτό το βήμα ολοκληρώνουμε τη λειτουργία της μηχανής αναζήτησης. Περιλαμβάνει:

1. **Γενικό Κώδικα Μηχανής Αναζήτησης**:
   - Συνδυάζει την επεξεργασία ερωτήματος (query processing) με την κατάταξη των αποτελεσμάτων (ranking).

2. **Υποενότητα 4α: Επεξεργασία Ερωτήματος**:
   - Αναλύει το ερώτημα και επιστρέφει τις πιθανές εγγραφές.

3. **Υποενότητα 4β: Κατάταξη Αποτελεσμάτων**:
   - Υλοποίηση 3 αλγορίθμων κατάταξης:
     - Κατάταξη με TF-IDF.
     - Κατάταξη με συχνότητα όρων.
     - Κατάταξη με συσχέτιση Cosine Similarity.

Κάθε υποενότητα περιλαμβάνει παραδείγματα με:
- Ένα έγκυρο ερώτημα: `angelos charisteas`.
- Ένα μη έγκυρο ερώτημα: `chris drousias`.

In [1]:
import json

def load_inverted_index(file_path):
    """Φόρτωση του ανεστραμμένου ευρετηρίου από αρχείο JSON με κανονικοποίηση."""
    with open(file_path, 'r', encoding='utf-8') as file:
        raw_index = json.load(file)
    # Κανονικοποίηση όλων των κλειδιών του ευρετηρίου σε πεζά
    normalized_index = {key.lower(): value for key, value in raw_index.items()}
    return normalized_index

def search_in_index(query, inverted_index):
    """Αναζήτηση ερωτήματος στο ανεστραμμένο ευρετήριο."""
    query = query.lower().strip()  # Κανονικοποίηση του ερωτήματος
    results = inverted_index.get(query, [])  # Αναζήτηση στο ευρετήριο
    return results

def simple_search_engine():
    """Απλή διεπαφή χρήστη για αναζήτηση."""
    inverted_index = load_inverted_index('inverted_index.json')  # Φόρτωση ευρετηρίου

    print("Μηχανή Αναζήτησης")
    print("Αναζητήστε οτιδήποτε ή πληκτρολογήστε 'exit' για έξοδο.")
    
    while True:
        query = input("\nΕισάγετε το ερώτημά σας: ").strip()
        if query.lower() == "exit":
            print("Έξοδος από τη μηχανή αναζήτησης...")
            break

        results = search_in_index(query, inverted_index)
        
        if results:
            print(f"Βρέθηκαν {len(results)} αποτελέσματα για το '{query}':")
            for result in results:
                print(f"- {result}")
        else:
            print(f"Δε βρέθηκαν αποτελέσματα για το '{query}'.")

if __name__ == "__main__":
    simple_search_engine()


Μηχανή Αναζήτησης
Αναζητήστε οτιδήποτε ή πληκτρολογήστε 'exit' για έξοδο.



Εισάγετε το ερώτημά σας:  angelos charisteas


Βρέθηκαν 20 αποτελέσματα για το 'angelos charisteas':
- {'team': 'Greece', 'match': '2001-03-28 - Greece vs Germany', 'minute': 20, 'own_goal': 'FALSE', 'penalty': 'FALSE'}
- {'team': 'Greece', 'match': '2001-10-06 - England vs Greece', 'minute': 36, 'own_goal': 'FALSE', 'penalty': 'FALSE'}
- {'team': 'Greece', 'match': '2003-04-02 - Northern Ireland vs Greece', 'minute': 4, 'own_goal': 'FALSE', 'penalty': 'FALSE'}
- {'team': 'Greece', 'match': '2003-04-02 - Northern Ireland vs Greece', 'minute': 55, 'own_goal': 'FALSE', 'penalty': 'FALSE'}
- {'team': 'Greece', 'match': '2003-06-11 - Greece vs Ukraine', 'minute': 86, 'own_goal': 'FALSE', 'penalty': 'FALSE'}
- {'team': 'Greece', 'match': '2004-06-16 - Greece vs Spain', 'minute': 66, 'own_goal': 'FALSE', 'penalty': 'FALSE'}
- {'team': 'Greece', 'match': '2004-06-25 - France vs Greece', 'minute': 65, 'own_goal': 'FALSE', 'penalty': 'FALSE'}
- {'team': 'Greece', 'match': '2004-07-04 - Portugal vs Greece', 'minute': 57, 'own_goal': 'FALSE',


Εισάγετε το ερώτημά σας:  chris drousias


Δε βρέθηκαν αποτελέσματα για το 'chris drousias'.



Εισάγετε το ερώτημά σας:  exit


Έξοδος από τη μηχανή αναζήτησης...


In [2]:
import json
import re
import unicodedata
from collections import defaultdict

# Φόρτωση του ανεστραμμένου ευρετηρίου
with open('inverted_index.json', 'r', encoding='utf-8') as f:
    raw_inverted_index = json.load(f)

# Κανονικοποίηση κλειδιών ευρετηρίου
inverted_index = {}
for key, docs in raw_inverted_index.items():
    normalized_key = unicodedata.normalize('NFKD', key.strip().lower())
    inverted_index[normalized_key] = docs

# Συνάρτηση για τη διασύνδεση και επεξεργασία του ερωτήματος
def process_query(query):
    # Εάν το ερώτημα δεν περιέχει Boolean τελεστές, επιστρέφεται ως ενιαίος όρος
    if not any(op in query.lower() for op in ["and", "or", "not"]):
        normalized_query = unicodedata.normalize('NFKD', query.strip().lower())
        return [normalized_query]

    # Διαφορετικά, γίνεται κανονική ανάλυση σε tokens
    tokens = query.lower().strip().split()
    normalized_tokens = [unicodedata.normalize('NFKD', token.strip()) for token in tokens]
    return normalized_tokens

# Λειτουργίες Boolean

def boolean_and(set1, set2):
    return set1.intersection(set2)

def boolean_or(set1, set2):
    return set1.union(set2)

def boolean_not(set1, all_docs):
    return all_docs - set1

# Επεξεργασία Boolean ερωτήματος
def process_boolean_query(query, inverted_index):
    all_docs = set()

    # Επαλήθευση και προσαρμογή της δομής του ανεστραμμένου ευρετηρίου
    for key, docs in inverted_index.items():
        if isinstance(docs, list):
            for doc in docs:
                if isinstance(doc, dict):
                    # Προσθέτουμε μόνο μοναδικούς αναγνωριστικούς δείκτες (π.χ., "match")
                    if "match" in doc:
                        all_docs.add(doc["match"])
                else:
                    all_docs.add(doc)

    tokens = process_query(query)

    # Στοίβα για αξιολόγηση
    result_stack = []
    operators = []

    for token in tokens:
        if token in {"and", "or", "not"}:
            operators.append(token)
        else:
            # Ανάκτηση της λίστας εμφάνισης για τον όρο
            posting_list = set()
            normalized_token = unicodedata.normalize('NFKD', token.strip().lower())

            # Εύρεση του όρου στο κανονικοποιημένο ευρετήριο (Partial Match)
            for key in inverted_index.keys():
                if normalized_token in key:
                    for doc in inverted_index[key]:
                        if isinstance(doc, dict) and "match" in doc:
                            posting_list.add(doc["match"])

            result_stack.append(posting_list)

            # Αξιολόγηση αν υπάρχει ο τελεστής NOT
            while operators and operators[-1] == "not":
                operators.pop()
                operand = result_stack.pop()
                result_stack.append(boolean_not(operand, all_docs))

    # Αξιολόγηση τελεστών AND/OR στη στοίβα
    while operators:
        operator = operators.pop(0)  # FIFO
        operand1 = result_stack.pop(0)
        operand2 = result_stack.pop(0)

        if operator == "and":
            result_stack.append(boolean_and(operand1, operand2))
        elif operator == "or":
            result_stack.append(boolean_or(operand1, operand2))

    return result_stack[0] if result_stack else set()

# Διεπαφή χρήστη για αναζήτηση
def query_interface():
    print("Μηχανή Αναζήτησης Boolean Ερωτημάτων")
    print("Εισάγετε το ερώτημά σας (χρησιμοποιήστε AND, OR, NOT):")
    while True:
        query = input("Ερώτημα: ")
        if query.lower() == "exit":
            break
        results = process_boolean_query(query, inverted_index)
        print(f"Έγγραφα που ταιριάζουν στο ερώτημα: {results}")

if __name__ == "__main__":
    query_interface()


Μηχανή Αναζήτησης Boolean Ερωτημάτων
Εισάγετε το ερώτημά σας (χρησιμοποιήστε AND, OR, NOT):


Ερώτημα:  angelos charisteas


Έγγραφα που ταιριάζουν στο ερώτημα: {'2001-03-28 - Greece vs Germany', '2004-11-17 - Greece vs Kazakhstan', '2007-06-06 - Greece vs Moldova', '2011-10-11 - Georgia vs Greece', '2008-06-18 - Greece vs Spain', '2006-10-11 - Bosnia Herzegovina vs Greece', '2004-06-16 - Greece vs Spain', '2004-07-04 - Portugal vs Greece', '2008-10-11 - Greece vs Moldova', '2007-10-13 - Greece vs Bosnia Herzegovina', '2008-10-15 - Greece vs Switzerland', '2001-10-06 - England vs Greece', '2003-04-02 - Northern Ireland vs Greece', '2005-03-30 - Greece vs Albania', '2003-06-11 - Greece vs Ukraine', '2004-06-25 - France vs Greece', '2008-09-06 - Luxembourg vs Greece'}


Ερώτημα:  chris drousias


Έγγραφα που ταιριάζουν στο ερώτημα: set()


Ερώτημα:  exit


In [3]:
import json
import re
import unicodedata
from collections import defaultdict, Counter
import math

# Φόρτωση του ανεστραμμένου ευρετηρίου
with open('inverted_index.json', 'r', encoding='utf-8') as f:
    raw_inverted_index = json.load(f)

# Κανονικοποίηση κλειδιών ευρετηρίου και κατακερματισμός σε λέξεις
inverted_index = defaultdict(list)
word_to_docs = defaultdict(set)

for key, docs in raw_inverted_index.items():
    normalized_key = unicodedata.normalize('NFKD', key.strip().lower())
    inverted_index[normalized_key] = docs

    for word in normalized_key.split():
        if isinstance(docs, list):
            for doc in docs:
                if isinstance(doc, dict) and "match" in doc:
                    word_to_docs[word].add((doc["match"], doc.get("minute", "N/A")))
                elif not isinstance(doc, dict):
                    word_to_docs[word].add((doc, "N/A"))
        elif isinstance(docs, dict) and "match" in docs:
            word_to_docs[word].add((docs["match"], docs.get("minute", "N/A")))
        elif not isinstance(docs, dict):
            word_to_docs[word].add((docs, "N/A"))

# Υπολογισμός TF-IDF
def compute_tf_idf(query, inverted_index, word_to_docs):
    """Υπολογισμός TF-IDF για την κατάταξη αποτελεσμάτων."""
    total_docs = sum(len(docs) if isinstance(docs, list) else 1 for docs in inverted_index.values())
    normalized_query = unicodedata.normalize('NFKD', query.strip().lower())

    scores = defaultdict(float)
    for term in normalized_query.split():
        if term in word_to_docs:
            doc_list = word_to_docs[term]
            doc_frequency = len(doc_list)
            idf = math.log(total_docs / (1 + doc_frequency))

            for doc_id, minute in doc_list:
                scores[(doc_id, minute)] += idf

    ranked_results = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked_results

# Vector Space Model (VSM)
def compute_vsm(query, inverted_index, word_to_docs):
    """Υπολογισμός VSM για την κατάταξη αποτελεσμάτων."""
    query_tokens = query.lower().strip().split()
    query_vector = Counter(query_tokens)

    doc_vectors = defaultdict(Counter)
    for term in query_tokens:
        if term in word_to_docs:
            for doc_id, minute in word_to_docs[term]:
                doc_vectors[(doc_id, minute)][term] += 1

    scores = {}
    for doc, vector in doc_vectors.items():
        dot_product = sum(query_vector[t] * vector[t] for t in query_tokens)
        query_norm = math.sqrt(sum(q ** 2 for q in query_vector.values()))
        doc_norm = math.sqrt(sum(v ** 2 for v in vector.values()))
        scores[doc] = dot_product / (query_norm * doc_norm) if query_norm and doc_norm else 0

    ranked_results = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked_results

# BM25
def compute_bm25(query, inverted_index, word_to_docs, k1=1.5, b=0.75):
    """Υπολογισμός BM25 για την κατάταξη αποτελεσμάτων."""
    total_docs = sum(len(docs) if isinstance(docs, list) else 1 for docs in inverted_index.values())
    avg_doc_len = sum(len(docs) for docs in inverted_index.values()) / total_docs

    scores = defaultdict(float)
    query_tokens = query.lower().strip().split()

    for term in query_tokens:
        if term in word_to_docs:
            doc_list = word_to_docs[term]
            doc_frequency = len(doc_list)
            idf = math.log((total_docs - doc_frequency + 0.5) / (doc_frequency + 0.5) + 1)

            for doc_id, minute in doc_list:
                doc_len = len(doc_list)
                tf = 1
                scores[(doc_id, minute)] += idf * ((tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (doc_len / avg_doc_len))))

    ranked_results = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked_results

# Διεπαφή χρήστη για αναζήτηση
def query_interface():
    print("Μηχανή Αναζήτησης με Πολλαπλούς Αλγορίθμους Κατάταξης")
    print("Επιλέξτε Αλγόριθμο: 1) TF-IDF  2) VSM  3) BM25")
    while True:
        choice = input("Επιλογή: ")
        if choice not in {"1", "2", "3"}:
            print("Μη έγκυρη επιλογή. Προσπαθήστε ξανά.")
            continue

        query = input("Ερώτημα: ")
        if query.lower() == "exit":
            break

        if choice == "1":
            ranked_results = compute_tf_idf(query, inverted_index, word_to_docs)
        elif choice == "2":
            ranked_results = compute_vsm(query, inverted_index, word_to_docs)
        elif choice == "3":
            ranked_results = compute_bm25(query, inverted_index, word_to_docs)

        if ranked_results:
            print("Αποτελέσματα κατάταξης:")
            for rank, ((doc_id, minute), score) in enumerate(ranked_results, start=1):
                print(f"{rank}. {doc_id} (Minute: {minute}) (Score: {score:.4f})")
        else:
            print("Δεν βρέθηκαν αποτελέσματα για το ερώτημά σας.")

if __name__ == "__main__":
    query_interface()


Μηχανή Αναζήτησης με Πολλαπλούς Αλγορίθμους Κατάταξης
Επιλέξτε Αλγόριθμο: 1) TF-IDF  2) VSM  3) BM25


Επιλογή:  1
Ερώτημα:  angelos charisteas


Αποτελέσματα κατάταξης:
1. 2007-06-06 - Greece vs Moldova (Minute: 30) (Score: 15.0599)
2. 2001-03-28 - Greece vs Germany (Minute: 20) (Score: 15.0599)
3. 2004-11-17 - Greece vs Kazakhstan (Minute: 24) (Score: 15.0599)
4. 2011-10-11 - Georgia vs Greece (Minute: 85) (Score: 15.0599)
5. 2006-10-11 - Bosnia Herzegovina vs Greece (Minute: 8) (Score: 15.0599)
6. 2008-10-11 - Greece vs Moldova (Minute: 31) (Score: 15.0599)
7. 2001-10-06 - England vs Greece (Minute: 36) (Score: 15.0599)
8. 2005-03-30 - Greece vs Albania (Minute: 33) (Score: 15.0599)
9. 2007-10-13 - Greece vs Bosnia Herzegovina (Minute: 10) (Score: 15.0599)
10. 2003-06-11 - Greece vs Ukraine (Minute: 86) (Score: 15.0599)
11. 2004-06-25 - France vs Greece (Minute: 65) (Score: 15.0599)
12. 2008-09-06 - Luxembourg vs Greece (Minute: 76) (Score: 15.0599)
13. 2003-04-02 - Northern Ireland vs Greece (Minute: 4) (Score: 15.0599)
14. 2008-10-11 - Greece vs Moldova (Minute: 51) (Score: 15.0599)
15. 2004-07-04 - Portugal vs Greece (Minu

Επιλογή:  1
Ερώτημα:  chris drousias


Αποτελέσματα κατάταξης:
1. 2016-06-08 - New Zealand vs New Caledonia (Minute: 49) (Score: 6.3434)
2. 1999-07-24 - New Zealand vs United States (Minute: 90) (Score: 6.3434)
3. 2005-11-12 - Trinidad Tobago vs Bahrain (Minute: 76) (Score: 6.3434)
4. 2022-03-24 - New Zealand vs New Caledonia (Minute: 83) (Score: 6.3434)
5. 2013-07-09 - United States vs Belize (Minute: 12) (Score: 6.3434)
6. 2022-03-30 - Solomon Islands vs New Zealand (Minute: 39) (Score: 6.3434)
7. 2017-09-04 - Northern Ireland vs Czech Republic (Minute: 41) (Score: 6.3434)
8. 2022-03-21 - New Zealand vs Fiji (Minute: 73) (Score: 6.3434)
9. 1974-10-30 - Sweden vs Northern Ireland (Minute: 7) (Score: 6.3434)
10. 1971-05-12 - England vs Malta (Minute: 74) (Score: 6.3434)
11. 2008-06-14 - Puerto Rico vs Honduras (Minute: 32) (Score: 6.3434)
12. 2001-06-11 - New Zealand vs Solomon Islands (Minute: 32) (Score: 6.3434)
13. 2005-07-06 - Trinidad Tobago vs Honduras (Minute: 28) (Score: 6.3434)
14. 2017-03-25 - Fiji vs New Zealand 

...

Επιλογή:  3
Ερώτημα:  angelos charisteas


Αποτελέσματα κατάταξης:
1. 2007-06-06 - Greece vs Moldova (Minute: 30) (Score: 1.4101)
2. 2001-03-28 - Greece vs Germany (Minute: 20) (Score: 1.4101)
3. 2004-11-17 - Greece vs Kazakhstan (Minute: 24) (Score: 1.4101)
4. 2011-10-11 - Georgia vs Greece (Minute: 85) (Score: 1.4101)
5. 2006-10-11 - Bosnia Herzegovina vs Greece (Minute: 8) (Score: 1.4101)
6. 2008-10-11 - Greece vs Moldova (Minute: 31) (Score: 1.4101)
7. 2001-10-06 - England vs Greece (Minute: 36) (Score: 1.4101)
8. 2005-03-30 - Greece vs Albania (Minute: 33) (Score: 1.4101)
9. 2007-10-13 - Greece vs Bosnia Herzegovina (Minute: 10) (Score: 1.4101)
10. 2003-06-11 - Greece vs Ukraine (Minute: 86) (Score: 1.4101)
11. 2004-06-25 - France vs Greece (Minute: 65) (Score: 1.4101)
12. 2008-09-06 - Luxembourg vs Greece (Minute: 76) (Score: 1.4101)
13. 2003-04-02 - Northern Ireland vs Greece (Minute: 4) (Score: 1.4101)
14. 2008-10-11 - Greece vs Moldova (Minute: 51) (Score: 1.4101)
15. 2004-07-04 - Portugal vs Greece (Minute: 57) (Score

Επιλογή:  3
Ερώτημα:  chris drousias


Αποτελέσματα κατάταξης:
1. 2016-06-08 - New Zealand vs New Caledonia (Minute: 49) (Score: 0.1804)
2. 1999-07-24 - New Zealand vs United States (Minute: 90) (Score: 0.1804)
3. 2005-11-12 - Trinidad Tobago vs Bahrain (Minute: 76) (Score: 0.1804)
4. 2022-03-24 - New Zealand vs New Caledonia (Minute: 83) (Score: 0.1804)
5. 2013-07-09 - United States vs Belize (Minute: 12) (Score: 0.1804)
6. 2022-03-30 - Solomon Islands vs New Zealand (Minute: 39) (Score: 0.1804)
7. 2017-09-04 - Northern Ireland vs Czech Republic (Minute: 41) (Score: 0.1804)
8. 2022-03-21 - New Zealand vs Fiji (Minute: 73) (Score: 0.1804)
9. 1974-10-30 - Sweden vs Northern Ireland (Minute: 7) (Score: 0.1804)
10. 1971-05-12 - England vs Malta (Minute: 74) (Score: 0.1804)
11. 2008-06-14 - Puerto Rico vs Honduras (Minute: 32) (Score: 0.1804)
12. 2001-06-11 - New Zealand vs Solomon Islands (Minute: 32) (Score: 0.1804)
13. 2005-07-06 - Trinidad Tobago vs Honduras (Minute: 28) (Score: 0.1804)
14. 2017-03-25 - Fiji vs New Zealand 

Επιλογή:  1
Ερώτημα:  exit


## Βήμα 5: Αξιολόγηση της Μηχανής Αναζήτησης

Σε αυτό το βήμα αξιολογούμε την απόδοση των αλγορίθμων κατάταξης (TF-IDF, VSM, BM25). Χρησιμοποιούμε μετρικές Precision, Recall, και F1-Score, βασισμένες σε προκαθορισμένα queries και ground truth δεδομένα.

### Μεθοδολογία
1. Επιλέγουμε test queries και ορίζουμε τα αναμενόμενα αποτελέσματα (ground truth).
2. Εκτελούμε τα queries για κάθε αλγόριθμο κατάταξης.
3. Υπολογίζουμε:
   - **Precision**: Ποσοστό σχετικών εγγραφών μεταξύ αυτών που ανακτήθηκαν.
   - **Recall**: Ποσοστό σχετικών εγγραφών που ανακτήθηκαν σε σχέση με το σύνολο των σχετικών.
   - **F1-Score**: Συνδυασμός Precision και Recall.

In [1]:
import json
from text_processing import preprocess_text, preprocess_dataset, save_json_inline
from inverted_index import inverted_index
from search_engine import load_inverted_index, search_in_index
from query_processing import process_query, process_boolean_query
from ranking import compute_tf_idf, compute_vsm, compute_bm25, word_to_docs
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Βήμα 1: Προεπεξεργασία dataset
print("Step 1: Preprocessing Dataset")
input_file = 'goalscorers.json'
output_file = 'goalscorers_processed.json'

data = preprocess_dataset(json.load(open(input_file, 'r', encoding='utf-8')))
save_json_inline(data, output_file)
print(f"Dataset processed and saved to {output_file}")

# Βήμα 2: Δημιουργία και αποθήκευση αντεστραμμένου ευρετηρίου
print("Step 2: Building Inverted Index")
index = inverted_index
for idx, record in enumerate(data):
    scorer = record["scorer"].lower()
    index[scorer].append(record)

with open("inverted_index.json", "w", encoding="utf-8") as outfile:
    json.dump(index, outfile, ensure_ascii=False, indent=4)
print("Inverted index built and saved to inverted_index.json")

# Βήμα 3: Επεξεργασία ερωτήματος και αναζήτηση
print("Step 3: Query Processing and Search")
inverted_index = load_inverted_index("inverted_index.json")
query = "angelos charisteas"

# Κανονικοποίηση ερωτήματος
normalized_query = query.lower().strip()
results = search_in_index(normalized_query, inverted_index)
print(f"Results for query '{query}': {results}")

# Βήμα 4: Αποτελέσματα κατάταξης
print("Step 4: Ranking Results")

ranked_results_tfidf = compute_tf_idf(normalized_query, inverted_index, word_to_docs)
print("TF-IDF Ranking Results:", ranked_results_tfidf)

ranked_results_vsm = compute_vsm(normalized_query, inverted_index, word_to_docs)
print("VSM Ranking Results:", ranked_results_vsm)

ranked_results_bm25 = compute_bm25(normalized_query, inverted_index, word_to_docs)
print("BM25 Ranking Results:", ranked_results_bm25)

# Βήμα 5: Αξιολόγηση συστήματος
print("Step 5: Evaluation")
ground_truth = [
    {
        "query": "angelos charisteas",
        "relevant_docs": ["Angelos Charisteas", "Angelos Charisteas"]
    }
]

evaluations = []
for item in ground_truth:
    query = item["query"]
    relevant_docs = set(item["relevant_docs"])

    results = search_in_index(query, inverted_index)
    retrieved_docs = [doc['scorer'] for doc in results]

    # Μετρικές
    y_true = [1 if doc in relevant_docs else 0 for doc in retrieved_docs]
    y_pred = [1] * len(retrieved_docs)

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    evaluations.append({
        "query": query,
        "precision": precision,
        "recall": recall,
        "f1": f1
    })

# Εκτύπωση και αποθήκευση αξιολογήσεων
for eval_result in evaluations:
    print(f"Evaluation for query '{eval_result['query']}': Precision={eval_result['precision']:.2f}, Recall={eval_result['recall']:.2f}, F1-Score={eval_result['f1']:.2f}")

with open("evaluation_results.json", "w", encoding="utf-8") as eval_file:
    json.dump(evaluations, eval_file, indent=4)
print("Evaluation results saved to evaluation_results.json")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\johnp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Η αποθήκευση ολοκληρώθηκε. Το αρχείο αποθηκεύτηκε στη διαδρομή: goalscorers_processed.json
Ο ανεστραμμένος πίνακας δημιουργήθηκε και αποθηκεύτηκε στο inverted_index.json.
Step 1: Preprocessing Dataset
Dataset processed and saved to goalscorers_processed.json
Step 2: Building Inverted Index
Inverted index built and saved to inverted_index.json
Step 3: Query Processing and Search
Results for query 'angelos charisteas': [{'date': '2001-03-28', 'home_team': 'Greece', 'away_team': 'Germany', 'team': 'Greece', 'scorer': 'Angelos Charisteas', 'minute': 20, 'own_goal': 'FALSE', 'penalty': 'FALSE'}, {'date': '2001-10-06', 'home_team': 'England', 'away_team': 'Greece', 'team': 'Greece', 'scorer': 'Angelos Charisteas', 'minute': 36, 'own_goal': 'FALSE', 'penalty': 'FALSE'}, {'date': '2003-04-02', 'home_team': 'Northern Ireland', 'away_team': 'Greece', 'team': 'Greece', 'scorer': 'Angelos Charisteas', 'minute': 4, 'own_goal': 'FALSE', 'penalty': 'FALSE'}, {'date': '2003-04-02', 'home_team': 'North

#### Συμπερασματικά, το main.py δεν λειτουργεί μόνο ως διεπαφή χρήστη, αλλά και ως εργαλείο για την αξιολόγηση της απόδοσης.